In [2]:
from IPython import display
import torch
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

In [3]:
# 28 * 28 = 784
# 类别：10
num_inputs = 784
num_outputs = 10
W = torch.normal(0, 0.01, size=(num_inputs, num_outputs),requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

In [4]:
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X.sum(0, keepdim=True), X.sum(1, keepdim=True)

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

In [5]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition

In [6]:
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X, X_prob, X_prob.sum(1, keepdim=True)

(tensor([[ 1.0369, -0.5034, -0.1681,  0.4725, -0.2172],
         [-1.9720, -0.7576,  0.6629,  0.4206, -0.8656]]),
 tensor([[0.4223, 0.0905, 0.1266, 0.2401, 0.1205],
         [0.0310, 0.1044, 0.4320, 0.3390, 0.0937]]),
 tensor([[1.0000],
         [1.0000]]))

In [7]:
def net(X):
    # matmul矩阵乘法（标量与标量的乘法，向量与向量的点积，矩阵与矩阵的乘法）
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

In [8]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

tensor([0.1000, 0.5000])

In [9]:
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])

cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

In [10]:
"""
先看y_hat是不是一个多列矩阵（是不是多类别预测），如果是
则找到y_hat每一行的最大值对应的索引，与标签比对，看看是否准确
"""
def accuracy(y_hat, y):  #@save
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        """argmax返回的是目标函数取得最大值的参数，就是x=x0时f(x)取得最大值，则x0=argmax(f(x))"""
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [11]:
accuracy(y_hat, y) / len(y)

0.5

In [12]:
class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
# 迭代器
def evaluate_accuracy(net, data_iter):  #@save
    """计算在指定数据集上的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval() # 模型设置：评估模式
    metric = Accumulator(2)  # 正确预测数，预测总数
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [13]:
evaluate_accuracy(net, test_iter)

0.1374